In [1]:
import librosa
import wave
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [2]:
DATA_PATH = ("./data/")
X_train = []
X_test = []
Y_train = []
Y_test = []
total_class = 0

In [6]:
def load_wave_generator(path): 
       
    batch_waves = []
    labels = []
    X_data = []
    Y_label = []    
    global X_train, X_test, Y_train, Y_test, total_class
    
    folders = os.listdir(path)

    for folder in folders:
        if not os.path.isdir(path):continue #폴더가 아니면 continue                   
        files = os.listdir(path+"/"+folder)        
        #폴더 이름과 그 폴더에 속하는 파일 갯수 출력
        for wav in files:
            if not wav.endswith(".wav"):continue
            elif wav.endswith('.wav'):               
                #print("Filename :",wav)#.wav 파일이 아니면 continue
                y, sr = librosa.load(path+"/"+folder+"/"+wav)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T
              
                X_data.extend(mfcc)
               # print(len(mfcc))
                
                label = [0 for i in range(len(folders))]
                label[total_class] = 1
                
                for i in range(len(mfcc)):
                    Y_label.append(label)
                #print(Y_label)
            else : 
                y, sr = librosa.load(path+"/"+folder+"/"+mp3)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T
              
                X_data.extend(mfcc)
               # print(len(mfcc))
                
                label = [0 for i in range(len(folders))]
                label[total_class] = 1
                
                for i in range(len(mfcc)):
                    Y_label.append(label)
        total_class = total_class+1
    #end loop
#     print("X_data :",np.shape(X_data))
#     print("Y_label :",np.shape(Y_label))
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_data), np.array(Y_label))

    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./data.npy",xy)


In [7]:
total_class = 0

load_wave_generator(DATA_PATH)

print(total_class,"개의 클래스")

8 개의 클래스


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_val, X_test, Y_val, Y_test = train_test_split(X_test,Y_test,test_size = 0.8, random_state = 1)


print("X_train :",np.shape(X_train))
print("Y_train :",np.shape(Y_train))
print("X_test :",np.shape(X_test))
print("Y_test :",np.shape(Y_test))
print("X_val :",np.shape(X_val))
print("Y_val :",np.shape(Y_val))

X_train : (59663, 13)
Y_train : (59663, 8)
X_test : (15911, 13)
Y_test : (15911, 8)
X_val : (3977, 13)
Y_val : (3977, 8)


In [10]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

In [11]:
np.random.seed(777)

model = Sequential()

model.add(Dense(256,input_shape=(13,), activation = 'relu'))

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(total_class,activation='softmax'))

In [12]:
model.compile(loss='binary_crossentropy',optimizer = 'Adam',metrics=['accuracy'])
model.summary()
trained_model = model.fit(X_train, Y_train,
          batch_size=32,
          epochs=35,
          validation_data=(X_val, Y_val),
          shuffle=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               3584      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

In [13]:
model.save_weights("model0915.h5")

In [14]:
model.load_weights("model0915.h5")

In [15]:
res = model.evaluate(X_test, Y_test)

498/498 [==============================] - 0s 886us/step - loss: 0.1512 - accuracy: 0.7728


In [17]:
y, sr = librosa.load(os.path.join("./test_유진.wav"))

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

output = model.predict(X_test, steps =1)

In [18]:
'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
5 청하
6 정국
7 나
'''

res = np.sum(output, axis = 0)/np.sum(output)*100

if np.max(res) > 50 :
    print("result : ",np.argmax(res))
else :
    print("no result")
    print(res)

result :  7
